# import / 라이브러리 호출

In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor

# Fixed RandomSeed / 랜덤시드 고정

seed를 고정하지 않는다면 같은 코드라도 매번 다른 결과가 나오게됩니다.

항상 동일한 결과를 얻기 위해서 사용합니다.

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# Data Load / 데이터 불러오기

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [4]:
train.head()

,ID,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형,수요량
0,TRAIN_0000,1,쇼핑몰 15,212000,0,도시 6,지역 1,쇼핑몰 유형 2,명절혼합과일선물세트,28
1,TRAIN_0001,2,쇼핑몰 72,113000,0,도시 21,지역 1,쇼핑몰 유형 1,발효홍삼선물세트,27
2,TRAIN_0002,0,쇼핑몰 15,67000,0,도시 6,지역 1,쇼핑몰 유형 2,실속스팸선물세트,769
3,TRAIN_0003,1,쇼핑몰 13,206000,0,도시 12,지역 3,쇼핑몰 유형 1,자연산프리미엄버섯선물세트,27
4,TRAIN_0004,1,쇼핑몰 65,140000,0,도시 16,지역 2,쇼핑몰 유형 2,자연산새우선물세트,337


# Feature & Target Split / 독립변수, 종속변수로 나누기
모델을 학습시키기 위해서 독립변수(x)와 종속변수(y)로 나누어야 합니다.  

단, 분석에 활용될 수 없는 ID 열(column)은 제거하겠습니다.

In [5]:
train_x = train.drop(['ID', '수요량'], axis = 1)
train_y = train['수요량']

test_x = test.drop('ID', axis = 1)

# Data Preprocessing / 데이터 전처리   
데이터 전처리는 결측치 제거, 데이터 단위 변환 등 데이터에 여러 가지 처리를 해주는 것입니다. 

전처리를 함으로써 데이터 분석이 가능하기 때문에 데이터 분석에 있어서 반드시 필수적인 부분입니다.

이번 Baseline 코드에서 사용할 전처리 방법은 Label Encoding 입니다.  

Label Encoding은 문자열 데이터를 수치형 데이터로 변환합니다.    

수치형 데이터로 변환함으로써 데이터를 알고리즘에 입력할 수 있습니다.  

In [6]:
ordinal_features = ['쇼핑몰 구분', '도시 유형', '지역 유형', '쇼핑몰 유형', '선물 유형']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train_x[feature])
    train_x[feature] = le.transform(train_x[feature])

    # train데이터에서 존재하지 않았던 값이 test 데이터에 존재할 수도 있습니다.
    # 따라서 test 데이터를 바로 변형시키지 않고 고유값을 확인후 test 데이터를 변환합니다.
    # Data Leakage를 발생시키지 않기 위함이니, 반드시 주의해주세요.
    for label in np.unique(test_x[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[feature] = le.transform(test_x[feature])

/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version.

# Regression Model Definition / 회귀 모델 정의

베이스라인에서 사용할 모델은 의사결정나무(Decision Tree) 입니다.  

아래 링크에서 의사 결정 나무의 개념을 확인해보세요.

https://dacon.io/forum/403509?dtype=recent

In [7]:
model = DecisionTreeRegressor(random_state = 42)

# Model fitting / 모델 학습

In [8]:
model.fit(train_x, train_y)

/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is de

DecisionTreeRegressor(random_state=42)

# Inference / 추론

In [9]:
preds = model.predict(test_x)

/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/syshin/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


# submit / 제출

In [13]:
submission = pd.read_csv('./data/sample_submission.csv')

In [14]:
submission['수요량'] = preds
submission.head()

,ID,수요량
0,TEST_0000,55.0
1,TEST_0001,26.0
2,TEST_0002,378.0
3,TEST_0003,109.0
4,TEST_0004,271.0


In [15]:
submission.to_csv('./data/submit.csv', index = False)

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=150, max_depth=30, min_samples_split=5, min_samples_leaf=2, bootstrap=True, random_state=42)

# Train the model
rf_model.fit(X_train_standard, y_train)

# Predict on the validation set
y_pred_rf = rf_model.predict(X_val_standard)

# Calculate RMSE for the Random Forest model
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))


NameError: name 'X_train_standard' is not defined